In [1]:
import pandas as pd
import collections, numpy,os
from collections import Counter
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re

In [5]:
df = pd.read_excel('/content/Amal Twitts.xlsx')

##Cleaning

In [6]:
!git clone https://github.com/UBC-NLP/aoc_id.git

Cloning into 'aoc_id'...
remote: Enumerating objects: 72, done.
remote: Total 72 (delta 0), reused 0 (delta 0), pack-reused 72
Unpacking objects: 100% (72/72), done.


In [7]:
from aoc_id import text_normalization

In [8]:
df['norm_text'] = df.text.apply(lambda text: text_normalization.clean_unicode(str(text)))

In [9]:
# to remove retweeted tweets as they cannot be lnger than 140 chars and usulally contain repeated info 
match = 'RT'
print('Number of Tweets before removing rows contain unwnated word: ',df.shape)
df = df[df['text'].str.contains(match)==False]
print('Number of Tweets after removing rows contain unwnated word: ',df.shape)

Number of Tweets before removing rows contain unwnated word:  (140649, 13)
Number of Tweets after removing rows contain unwnated word:  (109293, 13)


In [10]:
# to remove duplicates from clean text column
print('Number of Tweets before removing duplicates: ',df.shape)
df = df.drop_duplicates(['norm_text'])
print('Number of Tweets after removing duplicates: ',df.shape)


Number of Tweets before removing duplicates:  (109293, 13)
Number of Tweets after removing duplicates:  (86413, 13)


In [11]:
# function to find URL
def ck_url(x):
  re_equ = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
  ck_url = re.findall(re_equ, x)
  if ck_url:
      return True
  else:
      return False  

In [12]:
# to remove tweets that contain URL
print('Number of Tweets before removing URLs: ',df.shape)
df = df[df['text'].apply(ck_url) == False]
print('Number of Tweets after removing URLs: ',df.shape)


Number of Tweets before removing URLs:  (86413, 13)
Number of Tweets after removing URLs:  (47652, 13)


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47652 entries, 0 to 140644
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author_id           47652 non-null  object 
 1   username            47651 non-null  object 
 2   author_followers    47652 non-null  int64  
 3   author_tweets       47652 non-null  object 
 4   author_description  37170 non-null  object 
 5   author_location     23675 non-null  object 
 6   tweet_id            47652 non-null  float64
 7   text                47652 non-null  object 
 8   created_at          47651 non-null  object 
 9   retweets            47650 non-null  float64
 10  replies             47650 non-null  float64
 11  likes               47650 non-null  float64
 12  norm_text           47652 non-null  object 
dtypes: float64(4), int64(1), object(8)
memory usage: 5.1+ MB


In [14]:

columns_to_drop = ['created_at','tweet_id','author_id','author_description','replies','author_location']
df = df.drop(columns=columns_to_drop)

In [15]:
df.dropna()

,username,author_followers,author_tweets,text,retweets,likes,norm_text
0,raumunz,3070,1054648,الحج المبرور ليس له جزاء إلا الجنة#السيرة_كأن...,222.0,0.0,الحج المبرور ليس له جزاء الا الجنه السيره كان...
1,hesheey,713,19244,رئيس ساحل العاج يرفض البروتوكول السعودي في الحج,161.0,0.0,رئيس ساحل العاج يرفض البروتوكول السعودى فى الحج
2,ealam_itti,22300,22243,ما تقوم به بلادنا في خدمة حجاج بيت الله الحرا...,692.0,0.0,ما تقوم به بلادنا فى خدمه حجاج بيت الله الحرا...
3,3krstzzm,54,7054,شكرا_قيادتنا_على_نجاح_الحج الحمدلله حتى يبلغ ا...,2681.0,0.0,شكرا قيادتنا على نجاح الحج الحمدلله حتى يبلغ ا...
4,othman111mo,3563,146035,الاتحاد الأوروبي: السعودية نجحت بإدارة موسم #...,63.0,0.0,الاتحاد الاوروبى السعوديه نجحت باداره موسم ال...
...,...,...,...,...,...,...,...
140631,cxi_52,7,128,@MtjrRyf @n_2ad ادخل على وازرة الحج واحجز عالذ...,0.0,0.0,ادخل على وازره الحج واحجز عالذاكرين
140632,coachshreif3ouf,42009,114342,اليوم 8 ذي الحجة هو يوم التروية حيث تبدأ اول م...,21.0,231.0,اليوم ذى الحجه هو يوم الترويه حيث تبدا اول منا...
140638,LAljmazy,15,177,@kamalsinai كتب الله لنا ولكم الحج والعمرة مرة...,0.0,1.0,كتب الله لنا ولكم الحج والعمره مره بعد مره
140640,vilnca,158,2885,يازييييين مشاعر الحج 🥹🤍,0.0,1.0,يازيين مشاعر الحج


In [ ]:
df.to_csv('cleaned_data.csv')

**To predict sentiment analsis based on MAZAJAK model**

In [17]:
import requests
import json

In [ ]:
'''
This function offers the ability to predict the sentiment of a single sentence 
through the API, the sentiment is one of three classes (positive negative, neutral)
Input: 
        sentence(str): the input sentence of which the sentiment is to be predicted
Output:
        prediction(str): the sentiment of the given sentence 
'''


def predict(sentence):
    url = "http://mazajak.inf.ed.ac.uk:8000/api/predict"
    to_sent = {'data': sentence}
    data = json.dumps(to_sent,ensure_ascii=False).encode('utf8')
    headers = {'content-type': 'application/json'}
    # sending get request and saving the response as response object
    response = requests.post(url=url, data=data, headers=headers)

    prediction = json.loads(response.content)['data']

    return prediction


'''
This function offers the ability to predict the sentiment of a list of sentences
through the API, the sentiment is one of three classes (positive negative, neutral)
Input: 
        sent_lst(list of str): the input list of which the sentiment of its sentences is to be predicted
Output:
        prediction(list of str): the sentiments of the given sentences
'''


def predict_list(sent_lst):
    url = "http://mazajak.inf.ed.ac.uk:8000/api/predict_list"
    to_sent = {'data': sent_lst}
    data = json.dumps(to_sent)
    headers = {'content-type': 'application/json'}
    # sending get request and saving the response as response object
    response = requests.post(url=url, data=data, headers=headers)

    prediction = json.loads(response.content)['data']

    return prediction


In [ ]:
tweets = df['norm_text'].to_list()
labels = []
for tweet in tweets:
    labels.append(predict(tweet))

In [ ]:
df["auto_lable"]= labels

In [ ]:
df["auto_lable"].value_counts()

positive    23313
negative    15809
neutral      8530
Name: auto_lable, dtype: int64

In [ ]:
df

,username,author_followers,author_tweets,text,retweets,likes,norm_text,auto_lable
0,raumunz,3070,1054648,الحج المبرور ليس له جزاء إلا الجنة#السيرة_كأن...,222.0,0.0,الحج المبرور ليس له جزاء الا الجنه السيره كان...,positive
1,hesheey,713,19244,رئيس ساحل العاج يرفض البروتوكول السعودي في الحج,161.0,0.0,رئيس ساحل العاج يرفض البروتوكول السعودى فى الحج,neutral
2,ealam_itti,22300,22243,ما تقوم به بلادنا في خدمة حجاج بيت الله الحرا...,692.0,0.0,ما تقوم به بلادنا فى خدمه حجاج بيت الله الحرا...,positive
3,3krstzzm,54,7054,شكرا_قيادتنا_على_نجاح_الحج الحمدلله حتى يبلغ ا...,2681.0,0.0,شكرا قيادتنا على نجاح الحج الحمدلله حتى يبلغ ا...,positive
4,othman111mo,3563,146035,الاتحاد الأوروبي: السعودية نجحت بإدارة موسم #...,63.0,0.0,الاتحاد الاوروبى السعوديه نجحت باداره موسم ال...,neutral
...,...,...,...,...,...,...,...,...
140631,cxi_52,7,128,@MtjrRyf @n_2ad ادخل على وازرة الحج واحجز عالذ...,0.0,0.0,ادخل على وازره الحج واحجز عالذاكرين,neutral
140632,coachshreif3ouf,42009,114342,اليوم 8 ذي الحجة هو يوم التروية حيث تبدأ اول م...,21.0,231.0,اليوم ذى الحجه هو يوم الترويه حيث تبدا اول منا...,positive
140638,LAljmazy,15,177,@kamalsinai كتب الله لنا ولكم الحج والعمرة مرة...,0.0,1.0,كتب الله لنا ولكم الحج والعمره مره بعد مره,positive
140640,vilnca,158,2885,يازييييين مشاعر الحج 🥹🤍,0.0,1.0,يازيين مشاعر الحج,positive


In [ ]:
df.to_csv('Lable_data.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47652 entries, 0 to 140644
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   username          47651 non-null  object 
 1   author_followers  47652 non-null  int64  
 2   author_tweets     47652 non-null  object 
 3   text              47652 non-null  object 
 4   retweets          47650 non-null  float64
 5   likes             47650 non-null  float64
 6   norm_text         47652 non-null  object 
 7   auto_lable        47652 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 3.3+ MB


In [ ]:
df.loc[df['auto_lable']=='positive',:]

,username,author_followers,author_tweets,text,retweets,likes,norm_text,auto_lable
0,raumunz,3070,1054648,الحج المبرور ليس له جزاء إلا الجنة#السيرة_كأن...,222.0,0.0,الحج المبرور ليس له جزاء الا الجنه السيره كان...,positive
2,ealam_itti,22300,22243,ما تقوم به بلادنا في خدمة حجاج بيت الله الحرا...,692.0,0.0,ما تقوم به بلادنا فى خدمه حجاج بيت الله الحرا...,positive
3,3krstzzm,54,7054,شكرا_قيادتنا_على_نجاح_الحج الحمدلله حتى يبلغ ا...,2681.0,0.0,شكرا قيادتنا على نجاح الحج الحمدلله حتى يبلغ ا...,positive
6,rafa_XX8,522,2135,أحب أسمع قصص الحج من كبار السن أو من الأجيال ا...,1.0,2.0,احب اسمع قصص الحج من كبار السن او من الاجيال ا...,positive
7,SaadFalbuainain,2771,9986,نجاح منقطع النظير، يبعث على الفخر والاعتزاز، ...,5.0,0.0,نجاح منقطع النظير يبعث على الفخر والاعتزاز بك...,positive
...,...,...,...,...,...,...,...,...
140628,yhya_alameer,11231,25807,@Ali202223865001 مبروك الحج تقبل الله طاعتكم,0.0,1.0,مبروك الحج تقبل الله طاعتكم,positive
140632,coachshreif3ouf,42009,114342,اليوم 8 ذي الحجة هو يوم التروية حيث تبدأ اول م...,21.0,231.0,اليوم ذى الحجه هو يوم الترويه حيث تبدا اول منا...,positive
140638,LAljmazy,15,177,@kamalsinai كتب الله لنا ولكم الحج والعمرة مرة...,0.0,1.0,كتب الله لنا ولكم الحج والعمره مره بعد مره,positive
140640,vilnca,158,2885,يازييييين مشاعر الحج 🥹🤍,0.0,1.0,يازيين مشاعر الحج,positive


In [ ]:
df.loc[df['auto_lable']=='negative',:]

,username,author_followers,author_tweets,text,retweets,likes,norm_text,auto_lable
23,aziz87_abdul,21,1702,طبعا بالنسبة للاكل .. هذي لها عقود والمخالفات...,0.0,0.0,طبعا بالنسبه للاكل هذى لها عقود والمخالفات يت...,negative
27,majedd981,209,35597,ما نكبنا غير التهريب و #تاشيرة_زائر \nيا ليت ...,16.0,0.0,ما نكبنا غير التهريب تاشيره زائر يا ليت السنه ...,negative
31,alqaiser95,2016,60417,مره بالعمر و المقتدر مافي اشكاليه حسب الاستطا...,1.0,0.0,مره بالعمر المقتدر مافى اشكاليه حسب الاستطاعه...,negative
36,alfejraljded2,915,27582,: اقالة فريد عمادي باتت مطلب شعبي كل سنه الناس...,132.0,0.0,اقاله فريد عمادى باتت مطلب شعبى كل سنه الناس ...,negative
38,My_Angel1992,0,80,هذا من المفسدين ما أحترم حرمة المكان ولا مناسك...,0.0,2.0,هذا من المفسدين ما احترم حرمه المكان ولا مناسك...,negative
...,...,...,...,...,...,...,...,...
140545,wasekballah,8327,43562,ما يفعله ابن سلمان فى مناسك الحج يؤكد الحاجة إ...,0.0,2.0,ما يفعله ابن سلمان فى مناسك الحج يكد الحاجه ال...,negative
140557,sss08423144,36,4608,@Yaser46578913 @AJELNEWS24 غير صحيح بالعكس بيق...,1.0,0.0,غير صحيح بالعكس بيقل العرض وبيرتفع العقار وخل...,negative
140567,alharbi_muslih,371,1836,@ALMRISEUL اللي نقرأه منك ابعد مايكون عن المرا...,0.0,0.0,اللى نقراه منك ابعد مايكون عن المراجل اطفال ف...,negative
140571,rashed_fahad9,1134,15646,يقول ناصر خسرو في رحلته إلى الحج ذلك حين مرَّ ...,0.0,0.0,يقول ناصر خسرو فى رحلته الى الحج ذلك حين مر با...,negative


In [ ]:
df.loc[df['auto_lable']=='neutral',:]

,username,author_followers,author_tweets,text,retweets,likes,norm_text,auto_lable
1,hesheey,713,19244,رئيس ساحل العاج يرفض البروتوكول السعودي في الحج,161.0,0.0,رئيس ساحل العاج يرفض البروتوكول السعودى فى الحج,neutral
4,othman111mo,3563,146035,الاتحاد الأوروبي: السعودية نجحت بإدارة موسم #...,63.0,0.0,الاتحاد الاوروبى السعوديه نجحت باداره موسم ال...,neutral
5,nadeer5661333,71,1407,أثيوبي يحاول خطف دباب لعسكري في موسم الحج ويفز...,3714.0,0.0,اثىوبى يحاول خطف دباب لعسكرى فى موسم الحج ويفز...,neutral
9,xi0a3,1224,22150,#طريق_مكة\nالسعودية تنهض داخليا وفق منهجيات ال...,271.0,0.0,طريق مكه السعوديه تنهض داخليا وفق منهجيات الر...,neutral
25,dadahh87,1266,90262,\nـ وزير الداخلية لـ #خادم_الحرمين: دعمكم المب...,139.0,0.0,وزير الداخليه لـ خادم الحرمين دعمكم المباشر وت...,neutral
...,...,...,...,...,...,...,...,...
140527,Ali2030020300,272,8345,#منى\nأسماء أيام الحج الستة :\n\n٨ يوم التروية...,0.0,2.0,منى اسماء ايام الحج السته يوم الترويه يوم ع...,neutral
140539,Ali2030020300,272,8345,#يوم_التروية\nأسماء أيام الحج الستة :\n\n٨ يوم...,0.0,1.0,يوم الترويه اسماء ايام الحج السته يوم التروي...,neutral
140572,anees898,3276,1408,ومن أعمال يوم #التروية: الصلاة يصلي الحجاج الص...,0.0,0.0,ومن اعمال يوم الترويه الصلاه ىصلى الحجاج الصلو...,neutral
140579,nsns0r,64,394,@Tea4life4 المواعيد تتحدث كل ساعتين وثلاث .. خ...,0.0,0.0,المواعيد تتحدث كل ساعتين وثلاث خليك المواقع ب...,neutral


**To predict sentiment analsis based on Camels model**

In [22]:
!pip install camel_tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 114 kB 5.1 MB/s 
     |████████████████████████████████| 418 kB 39.4 MB/s 
     |████████████████████████████████| 197 kB 50.1 MB/s 
     |████████████████████████████████| 4.7 MB 35.3 MB/s 
     |████████████████████████████████| 6.6 MB 50.9 MB/s 
     |████████████████████████████████| 596 kB 51.5 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
  Created wheel for camel-kenlm: filename=camel_kenlm-2021.12.27-cp37-cp37m-linux_x86_64.whl size=2338948 sha256=b2b7b032bca1542411fd946505961981ed3a7af63ae6a6c779b5b3b3d5e26870
  Stored in directory: /root/.cache/pip/wheels/db/72/74/982f8c435f15b7feaf6dc8a03e212ff34e93f1f2d747059332
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=08958b0d7319a0645a495badf955fd72bd567dffe728d51c238a2e02e2408f42
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1

In [25]:
from camel_tools.sentiment import SentimentAnalyzer
sa = SentimentAnalyzer("CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")


In [26]:
df['camel_labe'] = df.norm_text.apply(lambda text: sa.predict(str(text)))

In [27]:
df['camel_labe']

0          [neutral]
1          [neutral]
2         [positive]
3         [positive]
4         [positive]
             ...    
140631     [neutral]
140632    [positive]
140638    [positive]
140640    [positive]
140644    [positive]
Name: camel_labe, Length: 47652, dtype: object

In [28]:
df

,username,author_followers,author_tweets,text,retweets,likes,norm_text,camel_labe
0,raumunz,3070,1054648,الحج المبرور ليس له جزاء إلا الجنة#السيرة_كأن...,222.0,0.0,الحج المبرور ليس له جزاء الا الجنه السيره كان...,[neutral]
1,hesheey,713,19244,رئيس ساحل العاج يرفض البروتوكول السعودي في الحج,161.0,0.0,رئيس ساحل العاج يرفض البروتوكول السعودى فى الحج,[neutral]
2,ealam_itti,22300,22243,ما تقوم به بلادنا في خدمة حجاج بيت الله الحرا...,692.0,0.0,ما تقوم به بلادنا فى خدمه حجاج بيت الله الحرا...,[positive]
3,3krstzzm,54,7054,شكرا_قيادتنا_على_نجاح_الحج الحمدلله حتى يبلغ ا...,2681.0,0.0,شكرا قيادتنا على نجاح الحج الحمدلله حتى يبلغ ا...,[positive]
4,othman111mo,3563,146035,الاتحاد الأوروبي: السعودية نجحت بإدارة موسم #...,63.0,0.0,الاتحاد الاوروبى السعوديه نجحت باداره موسم ال...,[positive]
...,...,...,...,...,...,...,...,...
140631,cxi_52,7,128,@MtjrRyf @n_2ad ادخل على وازرة الحج واحجز عالذ...,0.0,0.0,ادخل على وازره الحج واحجز عالذاكرين,[neutral]
140632,coachshreif3ouf,42009,114342,اليوم 8 ذي الحجة هو يوم التروية حيث تبدأ اول م...,21.0,231.0,اليوم ذى الحجه هو يوم الترويه حيث تبدا اول منا...,[positive]
140638,LAljmazy,15,177,@kamalsinai كتب الله لنا ولكم الحج والعمرة مرة...,0.0,1.0,كتب الله لنا ولكم الحج والعمره مره بعد مره,[positive]
140640,vilnca,158,2885,يازييييين مشاعر الحج 🥹🤍,0.0,1.0,يازيين مشاعر الحج,[positive]
